In [7]:
import pandas as pd
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import random
from operator import itemgetter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [8]:
# Load Train data 
df_train_prepared = pd.read_pickle('df_train_prepared')
 

train_payprice = df_train_prepared['payprice']
train_bidprice = df_train_prepared['bidprice']
train_click = df_train_prepared['click']

rtb_train = df_train_prepared.drop(['click','payprice','bidprice'], axis =1)
rtb_train.head()



,weekday,hour,region,city,adexchange,slotwidth,slotheight,slotprice,advertiser,useragent_encoded,IP_encoded,url_encoded,domain_encoded,slotid_encoded,slotvisibility_encoded,slotformat_encoded,creative_encoded,keypage_encoded,usertag_encoded
0,5,22,2,2,2,200,200,5,3427,31,253587,268016,21783,12845,2,0,104,0,744035
1,1,20,238,239,1,300,250,0,2821,29,282720,145905,728,50771,6,3,9,18,744035
2,3,13,40,41,2,250,250,5,3427,31,469429,262017,18426,9136,2,0,92,0,188248
3,6,23,1,1,1,160,600,0,1458,31,107833,658015,4770,50450,2,1,111,12,706361
4,5,6,216,233,2,728,90,133,2259,31,342191,157670,153,1517,8,3,84,18,744035


In [9]:
# Scale validation data for better prediction when using logistic 
standard_scaler = StandardScaler().fit(rtb_train)
train_x = standard_scaler.transform(rtb_train)


In [10]:
# Load validation data
df_val_prepared = pd.read_pickle('df_val_prepared')

val_payprice = df_val_prepared['payprice']
val_bidprice = df_val_prepared['bidprice']
val_click = df_val_prepared['click']

rtb_validation = df_val_prepared.drop(['click','payprice','bidprice'], axis =1)
rtb_validation.head()

FileNotFoundError: [Errno 2] No such file or directory: 'df_val_prepared'

In [ ]:
# Scale validation data for better prediction when using logistic 
standard_scaler = StandardScaler().fit(rtb_validation)
test_x = standard_scaler.transform(rtb_validation)


### Average CTR for Validation

In [ ]:
# THIS IS AVERAGE CTR FOR VALIDATION DATA 


val_click_1 = df_val_prepared[df_val_prepared['click'] == 1]
val_AvgCTR =  float("{0:.5f}".format(len(val_click_1)/len(df_val_prepared['click'])))
val_AvgCTR 

### Logistic Regression to Predict Probability of Click or Not Click

### LOGISTIC PREDICTION USING UNDER SAMPLING 

- Data is way too imblance only very little of click -> large prediction error
- Solution: UNDER SAMPLE
- Balance the click and non click -> improve the prediction for click or non click
 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

mean_class_size = int(pd.Series(train_click).value_counts().sum()/1500)
mean_class_size

ratio = {0: mean_class_size,
         1: mean_class_size}

rus = RandomUnderSampler (ratio = ratio, random_state = 0)
rus

train_x_rus, train_y_rus = rus.fit_sample(train_x, train_click)

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix 
 
log_reg = LogisticRegression()
clf2 = linear_model.LogisticRegression()
clf2.fit(train_x_rus, train_y_rus)

log2_prob  = clf2.predict_proba(test_x)
log2_pred = clf2.predict(test_x)

log2_error = sum(log2_pred != val_click) 
log2_correct = sum(log2_pred == val_click) 
print('Errors from Logistic with Under Sample:', log2_error)



In [ ]:
df_log2 = pd.DataFrame(columns=['probability_no_click','probability_click'], data = log2_prob)
df_log2['click'] = log2_pred
df_log2[840:845]

### RANDOM FOREST

Random Forest produce slightly better prediction

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(train_x_rus, train_y_rus)

### TRAIN PREDICTION WITH RANDOM FOREST

In [ ]:
# Prediction for train
train_tree_pred = tree_clf.predict(train_x)
train_tree_prob = tree_clf.predict_proba(train_x)

tree_error = sum(train_tree_pred != train_click) 
tree_correct = sum(train_tree_pred == train_click) 
print('Wrong Predictions from Decision Tree For Validation:', tree_error)


In [11]:
# Put in df
train_tree_predicted = pd.DataFrame(columns=['probability_no_click','probability_click'], data = train_tree_prob)

train_tree_predicted['click'] = train_tree_pred
train_tree_predicted.to_pickle('df_tree_prediction')
train_tree_predicted[840:845]
print (train_tree_predicted['probability_click'].mean())

NameError: name 'train_tree_prob' is not defined

### VALIDATION PREDICTION WITH RANDOM FOREST

In [12]:
# Prediction for validation
tree_pred = tree_clf.predict(test_x)
tree_prob = tree_clf.predict_proba(test_x)

tree_error = sum(tree_pred != val_click) 
tree_correct = sum(tree_pred == val_click) 
print('Wrong Predictions from Decision Tree For Validation:', tree_error)

NameError: name 'tree_clf' is not defined

In [17]:
# Put in df
df_tree_predicted = pd.DataFrame(columns=['probability_no_click','probability_click'], data = tree_prob)

df_tree_predicted['click'] = tree_pred 
df_tree_predicted.to_pickle('df_tree_prediction')
df_tree_predicted[840:845]

,probability_no_click,probability_click,click
840,0.613445,0.386555,0
841,0.100671,0.899329,1
842,0.613445,0.386555,0
843,0.613445,0.386555,0
844,0.613445,0.386555,0


In [18]:
# Mean square error for three models
#log1_rmse = np.sqrt(mean_squared_error(val_click, log1_pred))


log2_rmse = float("{0:.2f}".format(np.sqrt(mean_squared_error(val_click, log2_pred))))
tree_rmse = float("{0:.2f}".format(np.sqrt(mean_squared_error(val_click, tree_pred))))


model_errors = {'Logistic with Under Sample': [log2_rmse, log2_error, log2_correct], 'Decision Tree': [tree_rmse,tree_error, tree_correct]}
labels = ['RMSE', 'Misclassified', 'Correct Classified']

df_error = pd.DataFrame(model_errors, index = labels)
df_error



,Decision Tree,Logistic with Under Sample
RMSE,0.49,0.51
Misclassified,73985.00,79897.00
Correct Classified,229940.00,224028.00


### Linear Bidding 

basebid * pCRT/averageCTR 

pCTR = probability of click from Decision Tree Model 

In [19]:

pCTR_validation = df_tree_predicted['probability_click']
pCTR_validation.to_pickle('pCTR_validation')


In [20]:
pCTR_validation = pd.read_pickle('pCTR_validation')


In [22]:
pCTR_train = train_tree_predicted['probability_click']
pCTR_train.to_pickle('pCTR_train')

In [ ]:
pCTR_train = pd.read_pickle('pCTR_validation')